In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nibabel as nib
import numpy as np 
import matplotlib.pyplot as plt
import os
import random
import torch
import pandas as pd

In [ ]:
import tarfile
file = tarfile.open('../input/brats-2021-task1/BraTS2021_Training_Data.tar')

file.extractall('./brain_images')
file.close()

In [ ]:
PATIENT_ID=input()
T1_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t1.nii.gz"
T1CE_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t1ce.nii.gz"
T2_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_t2.nii.gz"
FLAIR_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_flair.nii.gz"
MASK_PATH=f"./brain_images/BraTS2021_{PATIENT_ID}/BraTS2021_{PATIENT_ID}_seg.nii.gz"
img_t1=nib.load(T1_PATH)
img_t1ce=nib.load(T1CE_PATH)
img_t2=nib.load(T2_PATH)
img_flair=nib.load(FLAIR_PATH)
img_mask=nib.load(MASK_PATH)
image_data_t1=img_t1.get_fdata()
image_data_t1ce=img_t1ce.get_fdata()
image_data_t2=img_t2.get_fdata()
image_data_flair=img_flair.get_fdata()
mask=img_mask.get_fdata()
print(img_t1.header)

In [ ]:
zero_matrix=np.zeros((240,240))
start=-1
end=1e9
nonEmptyMasks=[]
for i in range(0,155):
    if not np.array_equal(zero_matrix,mask[:,:,i]):
        if start==-1:
                start=i
        nonEmptyMasks.append(i)
        if i<end:
            end=i

def one_normal(x):
    if x==0:
        return 0
    else:
        return 1
plt.rcParams.update({'text.color': "red"})
LAYER=random.choice(nonEmptyMasks)
fig = plt.figure(figsize=(100,50))
rows = 5
cols=1
fig.add_subplot(rows, cols, 1)
cmask=np.vectorize(one_normal)(mask[:,:,LAYER])
plt.imshow(image_data_t1[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T1")
fig.add_subplot(rows, cols,2)
plt.imshow(image_data_t1ce[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T1CE")
fig.add_subplot(rows, cols, 3)
plt.imshow(image_data_t2[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("T2")
fig.add_subplot(rows, cols, 4)
plt.imshow(image_data_flair[:,:,LAYER]*cmask,cmap='gray')
plt.axis('off')
plt.title("FLAIR")
fig.add_subplot(rows, cols, 5)
plt.imshow(mask[:,:,LAYER],cmap='gray')
plt.axis('off')
plt.title("MASK")
fig.suptitle(f'Layer Number {LAYER}', fontsize=20)

In [ ]:
import torch
model_resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
model_vgg = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)

In [ ]:
labels=pd.read_csv('../input/mgmt-labels/train_labels_MGMT.csv')
CURRENT_FOLDER='./brain_images'
ALL_PATIENTS=[]
for f in os.listdir(CURRENT_FOLDER):
    path_to_dir=os.path.join(CURRENT_FOLDER,f)
    if os.path.isdir(path_to_dir):
        ALL_PATIENTS.append(f)
ALL_MGMT_PATIENTS=[str(label).zfill(5) for label in labels['BraTS21ID']]
print("Total Number of patients : ",len(ALL_PATIENTS))
print("Total Number of patients with MGMT Data : ",len(ALL_MGMT_PATIENTS))
print("Total Number of patients with MGMT Methylation Positive : ",len(labels[labels['MGMT_value']==1]))
print("Total Number of patients with MGMT Methylation Negative : ",len(labels[labels['MGMT_value']==0]))

In [ ]:
for i in ALL_MGMT_PATIENTS:
    if f'BraTS2021_{i}' not in ALL_PATIENTS:
        print(i)
    

In [ ]:
ALL_MGMT_PATIENTS=[i for i in ALL_MGMT_PATIENTS if f'BraTS2021_{i}' in ALL_PATIENTS]


In [ ]:
len(ALL_MGMT_PATIENTS)

In [ ]:
from tqdm.notebook import tqdm_notebook


In [ ]:
MAX_T1=-1
MIN_T1=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T1'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t1.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T1=max(MAX_T1,image_data.max())
    MIN_T1=min(MIN_T1,image_data.min())
print("Max Intensity in T1 Images : ",MAX_T1)
print("Min Intensity in T1 Images : ",MIN_T1)
MAX_T1CE=-1
MIN_T1CE=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T1CE'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t1ce.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T1CE=max(MAX_T1CE,image_data.max())
    MIN_T1CE=min(MIN_T1CE,image_data.min())
print("Max Intensity in T1 Images : ",MAX_T1CE)
print("Min Intensity in T1 Images : ",MIN_T1CE)


In [ ]:
MAX_T2=-1
MIN_T2=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='T2'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_t2.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_T2=max(MAX_T2,image_data.max())
    MIN_T2=min(MIN_T2,image_data.min())
print("Max Intensity in T2 Images : ",MAX_T2)
print("Min Intensity in T2 Images : ",MIN_T2)


In [ ]:
MAX_FLAIR=-1
MIN_FLAIR=1e9
for i in tqdm_notebook(ALL_MGMT_PATIENTS,desc='FLAIR'):
    img_t1=nib.load(f"./brain_images/BraTS2021_{i}/BraTS2021_{i}_flair.nii.gz")
    image_data=img_t1.get_fdata()
    MAX_FLAIR=max(MAX_FLAIR,image_data.max())
    MIN_FLAIR=min(MIN_FLAIR,image_data.min())
print("Max Intensity in FLAIR Images : ",MAX_FLAIR)
print("Min Intensity in FLAIR Images : ",MIN_FLAIR)

In [ ]:
print(type(image_data))

In [ ]:
## Minmax Normalisation to 0 to 255
def Normalise(A : np.ndarray ,MIN : np.float64,MAX : np.float64 ):
        A=A-MIN
        A=A/(MAX-MIN)
        A=A*255
        A[A>255.0]=255.0
        return A

In [ ]:
A=np.random.rand(240,240)*1000
print(A.max())
print(A.min())
A=Normalise(A,0,999)
print(A.max())
print(A.min())

In [ ]:
## Creting the models 
## Thought path 
    ## Store the intermediate results then process them 
    